# CNN UBS

In [3]:
#pip install keras-tuner --upgrade

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Normalization
%matplotlib inline
import seaborn as sns
from keras.models import Sequential 
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, InputLayer, SimpleRNN
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

from sklearn import (
    linear_model, metrics, neural_network, pipeline, model_selection
)
from sklearn.impute import SimpleImputer

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import TimeSeriesSplit

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
# %pip install -q -U keras-tuner
import keras_tuner as kt

In [5]:
df = pd.read_csv("UBS_Cleaned_Date.csv") #.iloc[2:, :].reset_index(drop = True)Netflix.date = pd.to_datetime(Netflix.date)
df = df.set_index("date")
df = df.drop(["Nas_total", 'Stock_total', 'Dow_total'],axis=1) # to remove duplicated columns
pd.DataFrame(df.isna().sum()).T

,UBS_x,UBS Financial Services Inc.,UBS Investment Bank,UBS Global Wealth Management,UBS Asset Management,Open,High,Low,Close,Volume,...,Dow_EMA_Move,Dow_EMA_Move_5,Dow_Disparity_Move,Dow_Disparity_s_Move,Dow_RSI_Move,target_1,target_2,target_3,target_4,target_5
0,18,18,18,18,18,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
df = df.dropna()
df = df[~(df.isin([np.inf, -np.inf]).any(axis=1))] 

In [7]:
df = df.drop(['target_1', 'target_2', 'target_4', 'target_5'], axis=1)
target_3 = df["target_3"]

#splitting into training sets 
column_indices = {name: i for i, name in enumerate(df.columns)}

n = len(df)
X_train = df[0:int(n*0.7)]
X_val = df[int(n*0.7):int(n*0.9)]
X_test = df[int(n*0.9):]

y_train = target_3[0:int(n*0.7)]
y_val = target_3[int(n*0.7):int(n*0.9)]
y_test = target_3[int(n*0.9):]

In [8]:
Mscaler = MinMaxScaler() # keeps binarys at zero and 1 :)

X_train = pd.DataFrame(Mscaler.fit_transform(X_train), columns = df.columns)
X_val = pd.DataFrame(Mscaler.fit_transform(X_val), columns = df.columns)
X_test = pd.DataFrame(Mscaler.fit_transform(X_test), columns = df.columns)

In [9]:
def kbest_creator(k, df, df_val, df_test):
    """
    returns data frame of principle componets of # of k best features 
    """
    # apply SelectKBest class to extract top 40 best features
    bestfeatures = SelectKBest(score_func=f_regression, k=k)
    best_fit = bestfeatures.fit(df, y_train)
    best_scores = pd.DataFrame(best_fit.scores_)
    best_columns = pd.DataFrame(df.columns)
    
    # concatenate the dataframes for better visualization
    features_score = pd.concat([best_columns, best_scores], axis=1)
    features_score.columns = ['Features', 'Score']  # naming the dataframe columns
    feats_kb = list(features_score.nlargest(k, 'Score').iloc[1:k]['Features'])

    pca = PCA().fit(df[feats_kb])
    pca_scores = np.cumsum(pca.explained_variance_ratio_) * 100
    res = next(x for x, val in enumerate(pca_scores) if val > 85)
    res = res + 1
    res
    
    pca_kb_1 = PCA(n_components = res).fit(df[feats_kb].to_numpy())
    df = pca_kb_1.transform(df[feats_kb].to_numpy())
    df_val = pca_kb_1.transform(df_val[feats_kb].to_numpy())
    df_test = pca_kb_1.transform(df_test[feats_kb].to_numpy())
    return df, df_val, df_test


In [10]:
X_train_kb_10, X_val_kb_10, X_test_kb_10 = kbest_creator(10, X_train, X_val, X_test) 
X_train_kb_25, X_val_kb_25, X_test_kb_25 = kbest_creator(25, X_train, X_val, X_test) 
X_train_kb_40, X_val_kb_40, X_test_kb_40 = kbest_creator(40, X_train, X_val, X_test) 
X_train_kb_55, X_val_kb_55, X_test_kb_55 = kbest_creator(55, X_train, X_val, X_test) 

/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:301: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:301: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:301: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:301: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms


# Model Testing

In [11]:
early_stopping_monitor = EarlyStopping(patience=4)

In [12]:
def df_to_X_y2(df, target, window_size=5):
  # converts to matrix of numpy arrays
  X = []
  y = []
  for i in range(len(df)-window_size): # length of data frame - window_size so it does't take empty values at the end, 
    # does force you to loose the last 5 values, could fix with padding
    row = [r for r in df[i:i+window_size]] # grabs row i and all rows above within the window size length
    X.append(row) # creates 3 dimentional array, (# obseravtions, # rows in window, # features)
    label = target[i+window_size] # pulls the target variable after the window, target varible needs to be column zero in this 
    y.append(label) # returns (N,) martix of targets i+window_length time periods away
  return np.array(X), np.array(y)

In [13]:
# converting to window format, in this case 5 periods
X_train_kb_40, train_5w = df_to_X_y2(X_train_kb_40,y_train)
X_val_kb_40, val_5w = df_to_X_y2(X_val_kb_40, y_val)
X_test_kb_40, test_5w = df_to_X_y2(X_test_kb_40,y_test) 

X_train_kb_10, _ = df_to_X_y2(X_train_kb_10,y_train)
X_val_kb_10, _ = df_to_X_y2(X_val_kb_10, y_val)
X_test_kb_10, _ = df_to_X_y2(X_test_kb_10,y_test) 

X_train_kb_25, _ = df_to_X_y2(X_train_kb_25,y_train)
X_val_kb_25, _ = df_to_X_y2(X_val_kb_25, y_val)
X_test_kb_25, _ = df_to_X_y2(X_test_kb_25,y_test) 

X_train_kb_55, _ = df_to_X_y2(X_train_kb_55,y_train)
X_val_kb_55, _ = df_to_X_y2(X_val_kb_55, y_val)
X_test_kb_55, _ = df_to_X_y2(X_test_kb_55,y_test) 

### Model Format 1

## Hyper Parameter tuned Model 1

In [14]:
# Model 1_1 - 1_4, using differnt k best pca variables

def model_builder_1_1(hp):
    n_steps = X_train_kb_10.shape[1]
    n_features = X_train_kb_10.shape[2]

    model1_1 = Sequential()

    hp_filters = hp.Int('units', min_value=4, max_value=128, step=4)
    hp_units = hp.Int('units', min_value=5, max_value=100, step=10)
    model1_1.add(Conv1D(filters=hp_filters, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

    model1_1.add(Flatten())
    model1_1.add(Dense(units = hp_units, activation='relu')) 
    model1_1.add(Dense(1, activation='sigmoid'))

    model1_1.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

    return model1_1

tuner = kt.Hyperband(model_builder_1_1,
                     objective='val_binary_accuracy',
                     max_epochs=30,
                     factor=3,
                     # directory='my_dir',    don't know what to do with this for now
                     # project_name='intro_to_kt' # sasme with this
                     )

tuner.search(X_train_kb_10, train_5w, epochs=50, validation_data=(X_val_kb_10, val_5w), callbacks = [early_stopping_monitor])

#model1_1.fit(X_train_kb_10, train_5w,epochs=30,  validation_data=(X_val_kb_10, val_5w), callbacks = [early_stopping_monitor])

Trial 28 Complete [00h 00m 02s]
val_binary_accuracy: 0.5517241358757019

Best val_binary_accuracy So Far: 0.5862069129943848
Total elapsed time: 00h 01m 14s
INFO:tensorflow:Oracle triggered exit


In [15]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
best_hps

In [16]:
model_1_1 = tuner.hypermodel.build(best_hps)
history = model_1_1.fit(X_train_kb_10, train_5w, epochs=30, validation_data=(X_val_kb_10, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30
17/17 [==============================] - 1s 23ms/step - loss: 0.6986 - binary_accuracy: 0.5058 - val_loss: 0.6851 - val_binary_accuracy: 0.5724
Epoch 2/30
17/17 [==============================] - 0s 20ms/step - loss: 0.6765 - binary_accuracy: 0.5849 - val_loss: 0.6904 - val_binary_accuracy: 0.5034
Epoch 3/30
17/17 [==============================] - 0s 8ms/step - loss: 0.6647 - binary_accuracy: 0.6390 - val_loss: 0.6961 - val_binary_accuracy: 0.5034
Epoch 4/30
17/17 [==============================] - 0s 8ms/step - loss: 0.6542 - binary_accuracy: 0.6409 - val_loss: 0.6983 - val_binary_accuracy: 0.4966
Epoch 5/30
17/17 [==============================] - 0s 8ms/step - loss: 0.6435 - binary_accuracy: 0.6564 - val_loss: 0.7070 - val_binary_accuracy: 0.5034


In [17]:
y_hat1_1 = model_1_1.predict(X_test_kb_10)
y_hat1_1 = y_hat1_1 > .5
metrics.accuracy_score(test_5w,y_hat1_1)

0.5142857142857142

## Normal Models

In [18]:
n_steps = X_train_kb_10.shape[1]
n_features = X_train_kb_10.shape[2]

model1_1 = Sequential()


model1_1.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model1_1.add(Flatten())
model1_1.add(Dense(units = 15, activation='relu')) 
model1_1.add(Dense(1, activation='sigmoid'))

model1_1.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model1_1.fit(X_train_kb_10, train_5w,epochs=30,  validation_data=(X_val_kb_10, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


17/17 [==============================] - 1s 21ms/step - loss: 0.6952 - binary_accuracy: 0.5077 - val_loss: 0.6925 - val_binary_accuracy: 0.5310
Epoch 2/30
17/17 [==============================] - 0s 9ms/step - loss: 0.6865 - binary_accuracy: 0.5463 - val_loss: 0.6891 - val_binary_accuracy: 0.5517
Epoch 3/30
17/17 [==============================] - 0s 9ms/step - loss: 0.6812 - binary_accuracy: 0.5714 - val_loss: 0.6896 - val_binary_accuracy: 0.5034
Epoch 4/30
17/17 [==============================] - 0s 9ms/step - loss: 0.6768 - binary_accuracy: 0.5830 - val_loss: 0.6895 - val_binary_accuracy: 0.5034
Epoch 5/30
17/17 [==============================] - 0s 6ms/step - loss: 0.6718 - binary_accuracy: 0.6004 - val_loss: 0.6898 - val_binary_accuracy: 0.5172
Epoch 6/30
17/17 [==============================] - 0s 8ms/step - loss: 0.6679 - binary_accuracy: 0.6004 - val_loss: 0.6886 - val_binary_accuracy: 0.5517
Epoch 7/30
17/17 [==============================] - 0s 7ms/step - loss: 0.6640 - binar

In [19]:
y_hat1_1 = model1_1.predict(X_test_kb_10)
y_hat1_1 = y_hat1_1 > .5
metrics.accuracy_score(test_5w,y_hat1_1)

0.44285714285714284

In [20]:
n_steps = X_train_kb_25.shape[1]
n_features = X_train_kb_25.shape[2]

model1_2 = Sequential()
model1_2.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model1_2.add(Flatten())
model1_2.add(Dense(25, activation='relu')) 
model1_2.add(Dense(1, activation='sigmoid'))

model1_2.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model1_2.fit(X_train_kb_25, train_5w,epochs=30,  validation_data=(X_val_kb_25, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


17/17 [==============================] - 2s 33ms/step - loss: 0.6970 - binary_accuracy: 0.4884 - val_loss: 0.6934 - val_binary_accuracy: 0.5379
Epoch 2/30
17/17 [==============================] - 0s 11ms/step - loss: 0.6803 - binary_accuracy: 0.5734 - val_loss: 0.6992 - val_binary_accuracy: 0.4966
Epoch 3/30
17/17 [==============================] - 0s 7ms/step - loss: 0.6703 - binary_accuracy: 0.6120 - val_loss: 0.7029 - val_binary_accuracy: 0.4828
Epoch 4/30
17/17 [==============================] - 0s 6ms/step - loss: 0.6630 - binary_accuracy: 0.6371 - val_loss: 0.7022 - val_binary_accuracy: 0.5103
Epoch 5/30
17/17 [==============================] - 0s 6ms/step - loss: 0.6541 - binary_accuracy: 0.6795 - val_loss: 0.7055 - val_binary_accuracy: 0.4966


In [21]:
y_hat1_2 = model1_2.predict(X_test_kb_25)
y_hat1_2 = y_hat1_2 > .5
metrics.accuracy_score(test_5w,y_hat1_2)

0.5142857142857142

In [22]:
n_steps = X_train_kb_40.shape[1]
n_features = X_train_kb_40.shape[2]

model1_3 = Sequential()
model1_3.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model1_3.add(Flatten())
model1_3.add(Dense(25, activation='relu')) 
model1_3.add(Dense(1, activation='sigmoid'))

model1_3.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model1_3.fit(X_train_kb_40, train_5w,epochs=30,  validation_data=(X_val_kb_40, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


17/17 [==============================] - 2s 50ms/step - loss: 0.7077 - binary_accuracy: 0.4749 - val_loss: 0.7041 - val_binary_accuracy: 0.4828
Epoch 2/30
17/17 [==============================] - 0s 12ms/step - loss: 0.6900 - binary_accuracy: 0.5328 - val_loss: 0.6985 - val_binary_accuracy: 0.4621
Epoch 3/30
17/17 [==============================] - 0s 12ms/step - loss: 0.6744 - binary_accuracy: 0.5830 - val_loss: 0.6988 - val_binary_accuracy: 0.4828
Epoch 4/30
17/17 [==============================] - 0s 6ms/step - loss: 0.6636 - binary_accuracy: 0.6139 - val_loss: 0.6969 - val_binary_accuracy: 0.5310
Epoch 5/30
17/17 [==============================] - 0s 8ms/step - loss: 0.6523 - binary_accuracy: 0.6467 - val_loss: 0.6983 - val_binary_accuracy: 0.5034
Epoch 6/30
17/17 [==============================] - 0s 4ms/step - loss: 0.6396 - binary_accuracy: 0.6931 - val_loss: 0.7034 - val_binary_accuracy: 0.4897
Epoch 7/30
17/17 [==============================] - 0s 12ms/step - loss: 0.6314 - bi

In [23]:
y_hat1_3 = model1_3.predict(X_test_kb_40)
y_hat1_3 = y_hat1_3 > .5
metrics.accuracy_score(test_5w,y_hat1_3)

0.5

In [24]:
n_steps = X_train_kb_55.shape[1]
n_features = X_train_kb_55.shape[2]

model1_4 = Sequential()
model1_4.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model1_4.add(Flatten())
model1_4.add(Dense(25, activation='relu')) 
model1_4.add(Dense(1, activation='sigmoid'))

model1_4.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model1_4.fit(X_train_kb_55, train_5w,epochs=30,  validation_data=(X_val_kb_55, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


17/17 [==============================] - 1s 14ms/step - loss: 0.7076 - binary_accuracy: 0.4768 - val_loss: 0.7057 - val_binary_accuracy: 0.4552
Epoch 2/30
17/17 [==============================] - 0s 11ms/step - loss: 0.6807 - binary_accuracy: 0.5502 - val_loss: 0.7010 - val_binary_accuracy: 0.4897
Epoch 3/30
17/17 [==============================] - 0s 8ms/step - loss: 0.6657 - binary_accuracy: 0.6120 - val_loss: 0.6996 - val_binary_accuracy: 0.5034
Epoch 4/30
17/17 [==============================] - 0s 8ms/step - loss: 0.6502 - binary_accuracy: 0.6544 - val_loss: 0.7020 - val_binary_accuracy: 0.5172
Epoch 5/30
17/17 [==============================] - 0s 21ms/step - loss: 0.6343 - binary_accuracy: 0.7259 - val_loss: 0.7035 - val_binary_accuracy: 0.5172
Epoch 6/30
17/17 [==============================] - 0s 9ms/step - loss: 0.6189 - binary_accuracy: 0.7259 - val_loss: 0.7058 - val_binary_accuracy: 0.5310
Epoch 7/30
17/17 [==============================] - 0s 8ms/step - loss: 0.6005 - bin

In [25]:
y_hat1_4 = model1_4.predict(X_test_kb_55)
y_hat1_4 = y_hat1_4 > .5
metrics.accuracy_score(test_5w,y_hat1_4)

0.4857142857142857

### Model 2, Less Filters

In [26]:
# Model 2_1 - 2_4, using differnt k best pca variables. reduced filters
n_steps = X_train_kb_10.shape[1]
n_features = X_train_kb_10.shape[2]

model2_1 = Sequential()
model2_1.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model2_1.add(Flatten())
model2_1.add(Dense(25, activation='relu')) 
model2_1.add(Dense(1, activation='sigmoid'))

model2_1.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model2_1.fit(X_train_kb_10, train_5w,epochs=30,  validation_data=(X_val_kb_10, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


17/17 [==============================] - 1s 12ms/step - loss: 0.7219 - binary_accuracy: 0.5135 - val_loss: 0.7187 - val_binary_accuracy: 0.4621
Epoch 2/30
17/17 [==============================] - 0s 15ms/step - loss: 0.7030 - binary_accuracy: 0.5154 - val_loss: 0.7119 - val_binary_accuracy: 0.5034
Epoch 3/30
17/17 [==============================] - 0s 5ms/step - loss: 0.6973 - binary_accuracy: 0.5000 - val_loss: 0.7101 - val_binary_accuracy: 0.4621
Epoch 4/30
17/17 [==============================] - 0s 5ms/step - loss: 0.6939 - binary_accuracy: 0.5039 - val_loss: 0.7079 - val_binary_accuracy: 0.4759
Epoch 5/30
17/17 [==============================] - 0s 6ms/step - loss: 0.6913 - binary_accuracy: 0.5135 - val_loss: 0.7062 - val_binary_accuracy: 0.4690
Epoch 6/30
17/17 [==============================] - 0s 5ms/step - loss: 0.6892 - binary_accuracy: 0.5193 - val_loss: 0.7044 - val_binary_accuracy: 0.4552
Epoch 7/30
17/17 [==============================] - 0s 4ms/step - loss: 0.6871 - bina

In [27]:
y_hat2_1 = model2_1.predict(X_test_kb_10)
y_hat2_1 = y_hat2_1 > .5
metrics.accuracy_score(test_5w,y_hat2_1)

0.5285714285714286

In [28]:
n_steps = X_train_kb_25.shape[1]
n_features = X_train_kb_25.shape[2]

model2_2 = Sequential()
model2_2.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model2_2.add(Flatten())
model2_2.add(Dense(25, activation='relu')) 
model2_2.add(Dense(1, activation='sigmoid'))

model2_2.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model2_2.fit(X_train_kb_25, train_5w,epochs=30,  validation_data=(X_val_kb_25, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


17/17 [==============================] - 1s 13ms/step - loss: 0.7078 - binary_accuracy: 0.4826 - val_loss: 0.6880 - val_binary_accuracy: 0.5241
Epoch 2/30
17/17 [==============================] - 0s 5ms/step - loss: 0.6975 - binary_accuracy: 0.4923 - val_loss: 0.6864 - val_binary_accuracy: 0.5310
Epoch 3/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6935 - binary_accuracy: 0.5174 - val_loss: 0.6855 - val_binary_accuracy: 0.5448
Epoch 4/30
17/17 [==============================] - 0s 5ms/step - loss: 0.6905 - binary_accuracy: 0.5367 - val_loss: 0.6863 - val_binary_accuracy: 0.5379
Epoch 5/30
17/17 [==============================] - 0s 7ms/step - loss: 0.6879 - binary_accuracy: 0.5386 - val_loss: 0.6872 - val_binary_accuracy: 0.5379
Epoch 6/30
17/17 [==============================] - 0s 4ms/step - loss: 0.6855 - binary_accuracy: 0.5541 - val_loss: 0.6893 - val_binary_accuracy: 0.5103
Epoch 7/30
17/17 [==============================] - 0s 4ms/step - loss: 0.6829 - binar

In [29]:
y_hat2_2 = model2_2.predict(X_test_kb_25)
y_hat2_2 = y_hat2_2 > .5
metrics.accuracy_score(test_5w,y_hat2_2)

0.45714285714285713

In [30]:
n_steps = X_train_kb_40.shape[1]
n_features = X_train_kb_40.shape[2]

model2_3 = Sequential()
model2_3.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model2_3.add(Flatten())
model2_3.add(Dense(25, activation='relu')) 
model2_3.add(Dense(1, activation='sigmoid'))

model2_3.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model2_3.fit(X_train_kb_40, train_5w,epochs=30,  validation_data=(X_val_kb_40, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


17/17 [==============================] - 1s 16ms/step - loss: 0.7110 - binary_accuracy: 0.5039 - val_loss: 0.7005 - val_binary_accuracy: 0.4966
Epoch 2/30
17/17 [==============================] - 0s 5ms/step - loss: 0.7019 - binary_accuracy: 0.4981 - val_loss: 0.7009 - val_binary_accuracy: 0.4759
Epoch 3/30
17/17 [==============================] - 0s 7ms/step - loss: 0.6978 - binary_accuracy: 0.5154 - val_loss: 0.7016 - val_binary_accuracy: 0.4759
Epoch 4/30
17/17 [==============================] - 0s 4ms/step - loss: 0.6929 - binary_accuracy: 0.5328 - val_loss: 0.7022 - val_binary_accuracy: 0.4690
Epoch 5/30
17/17 [==============================] - 0s 4ms/step - loss: 0.6892 - binary_accuracy: 0.5367 - val_loss: 0.7036 - val_binary_accuracy: 0.4828


In [31]:
y_hat2_3 = model2_3.predict(X_test_kb_40)
y_hat2_3 = y_hat2_3 > .5
metrics.accuracy_score(test_5w,y_hat2_3)

0.5285714285714286

In [32]:
n_steps = X_train_kb_55.shape[1]
n_features = X_train_kb_55.shape[2]

model2_4 = Sequential()
model2_4.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model2_4.add(Flatten())
model2_4.add(Dense(25, activation='relu')) 
model2_4.add(Dense(1, activation='sigmoid'))

model2_4.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model2_4.fit(X_train_kb_55, train_5w,epochs=30,  validation_data=(X_val_kb_55, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


17/17 [==============================] - 1s 14ms/step - loss: 0.7490 - binary_accuracy: 0.5000 - val_loss: 0.7178 - val_binary_accuracy: 0.4690
Epoch 2/30
17/17 [==============================] - 0s 6ms/step - loss: 0.7160 - binary_accuracy: 0.4942 - val_loss: 0.7006 - val_binary_accuracy: 0.5172
Epoch 3/30
17/17 [==============================] - 0s 3ms/step - loss: 0.7057 - binary_accuracy: 0.5019 - val_loss: 0.6973 - val_binary_accuracy: 0.5172
Epoch 4/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6995 - binary_accuracy: 0.5193 - val_loss: 0.6952 - val_binary_accuracy: 0.5103
Epoch 5/30
17/17 [==============================] - 0s 6ms/step - loss: 0.6939 - binary_accuracy: 0.5270 - val_loss: 0.6938 - val_binary_accuracy: 0.5103
Epoch 6/30
17/17 [==============================] - 0s 4ms/step - loss: 0.6892 - binary_accuracy: 0.5444 - val_loss: 0.6925 - val_binary_accuracy: 0.5172
Epoch 7/30
17/17 [==============================] - 0s 5ms/step - loss: 0.6846 - binar

In [33]:
y_hat2_4 = model2_4.predict(X_test_kb_55)
y_hat2_4 = y_hat2_4 > .5
metrics.accuracy_score(test_5w,y_hat2_4)

0.5

### Model 3, Adding layers

In [34]:
# Model 3_1 - 3_4, adding layers
n_steps = X_train_kb_10.shape[1]
n_features = X_train_kb_10.shape[2]

model3_1 = Sequential()
model3_1.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model3_1.add(Conv1D(filters=8, kernel_size=2, activation='relu')) 
model3_1.add(MaxPooling1D(pool_size=2))
model3_1.add(Flatten())
model3_1.add(Dense(8, activation='relu')) 
model3_1.add(Dense(1, activation='sigmoid'))

model3_1.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model3_1.fit(X_train_kb_10, train_5w,epochs=30,  validation_data=(X_val_kb_10, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


17/17 [==============================] - 1s 20ms/step - loss: 0.6925 - binary_accuracy: 0.4961 - val_loss: 0.6957 - val_binary_accuracy: 0.4690
Epoch 2/30
17/17 [==============================] - 0s 4ms/step - loss: 0.6914 - binary_accuracy: 0.4981 - val_loss: 0.6949 - val_binary_accuracy: 0.4828
Epoch 3/30
17/17 [==============================] - 0s 7ms/step - loss: 0.6906 - binary_accuracy: 0.5174 - val_loss: 0.6952 - val_binary_accuracy: 0.4897
Epoch 4/30
17/17 [==============================] - 0s 4ms/step - loss: 0.6901 - binary_accuracy: 0.5251 - val_loss: 0.6954 - val_binary_accuracy: 0.4897
Epoch 5/30
17/17 [==============================] - 0s 11ms/step - loss: 0.6894 - binary_accuracy: 0.5309 - val_loss: 0.6952 - val_binary_accuracy: 0.5034
Epoch 6/30
17/17 [==============================] - 0s 6ms/step - loss: 0.6889 - binary_accuracy: 0.5290 - val_loss: 0.6955 - val_binary_accuracy: 0.4759


In [35]:
y_hat3_1 = model3_1.predict(X_test_kb_10)
y_hat3_1 = y_hat3_1 > .5
metrics.accuracy_score(test_5w,y_hat3_1)

0.42857142857142855

In [36]:
n_steps = X_train_kb_25.shape[1]
n_features = X_train_kb_25.shape[2]

model3_2 = Sequential()
model3_2.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model3_2.add(Conv1D(filters=8, kernel_size=2, activation='relu')) 
model3_2.add(MaxPooling1D(pool_size=2))
model3_2.add(Flatten())
model3_2.add(Dense(25, activation='relu')) 
model3_2.add(Dense(1, activation='sigmoid'))

model3_2.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model3_2.fit(X_train_kb_25, train_5w,epochs=30,  validation_data=(X_val_kb_25, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


17/17 [==============================] - 1s 15ms/step - loss: 0.6929 - binary_accuracy: 0.5116 - val_loss: 0.6957 - val_binary_accuracy: 0.4690
Epoch 2/30
17/17 [==============================] - 0s 6ms/step - loss: 0.6882 - binary_accuracy: 0.5695 - val_loss: 0.6984 - val_binary_accuracy: 0.4207
Epoch 3/30
17/17 [==============================] - 0s 5ms/step - loss: 0.6855 - binary_accuracy: 0.5830 - val_loss: 0.6993 - val_binary_accuracy: 0.4276
Epoch 4/30
17/17 [==============================] - 0s 5ms/step - loss: 0.6804 - binary_accuracy: 0.6139 - val_loss: 0.7005 - val_binary_accuracy: 0.4414
Epoch 5/30
17/17 [==============================] - 0s 4ms/step - loss: 0.6758 - binary_accuracy: 0.6467 - val_loss: 0.7020 - val_binary_accuracy: 0.4621


In [37]:
y_hat3_2 = model3_2.predict(X_test_kb_25)
y_hat3_2 = y_hat3_2 > .5
metrics.accuracy_score(test_5w,y_hat3_2)

0.5142857142857142

In [38]:
n_steps = X_train_kb_40.shape[1]
n_features = X_train_kb_40.shape[2]

model3_3 = Sequential()
model3_3.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model3_3.add(Conv1D(filters=8, kernel_size=2, activation='relu')) 
model3_3.add(MaxPooling1D(pool_size=2))
model3_3.add(Flatten())
model3_3.add(Dense(25, activation='relu')) 
model3_3.add(Dense(1, activation='sigmoid'))

model3_3.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model3_3.fit(X_train_kb_40, train_5w,epochs=30,  validation_data=(X_val_kb_40, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


17/17 [==============================] - 1s 45ms/step - loss: 0.6956 - binary_accuracy: 0.5077 - val_loss: 0.6934 - val_binary_accuracy: 0.5241
Epoch 2/30
17/17 [==============================] - 0s 8ms/step - loss: 0.6913 - binary_accuracy: 0.5309 - val_loss: 0.6939 - val_binary_accuracy: 0.4897
Epoch 3/30
17/17 [==============================] - 0s 7ms/step - loss: 0.6891 - binary_accuracy: 0.5598 - val_loss: 0.6947 - val_binary_accuracy: 0.4897
Epoch 4/30
17/17 [==============================] - 0s 8ms/step - loss: 0.6865 - binary_accuracy: 0.5637 - val_loss: 0.6954 - val_binary_accuracy: 0.4966
Epoch 5/30
17/17 [==============================] - 0s 6ms/step - loss: 0.6835 - binary_accuracy: 0.5985 - val_loss: 0.6970 - val_binary_accuracy: 0.4828


In [39]:
y_hat3_3 = model3_3.predict(X_test_kb_40)
y_hat3_3 = y_hat3_3 > .5
metrics.accuracy_score(test_5w,y_hat3_3)

0.5285714285714286

In [40]:
n_steps = X_train_kb_55.shape[1]
n_features = X_train_kb_55.shape[2]

model3_4 = Sequential()
model3_4.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model3_4.add(Conv1D(filters=8, kernel_size=2, activation='relu')) 
model3_4.add(MaxPooling1D(pool_size=2))
model3_4.add(Flatten())
model3_4.add(Dense(25, activation='relu')) 
model3_4.add(Dense(1, activation='sigmoid'))

model3_4.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model3_4.fit(X_train_kb_55, train_5w,epochs=30,  validation_data=(X_val_kb_55, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


17/17 [==============================] - 1s 12ms/step - loss: 0.6964 - binary_accuracy: 0.4923 - val_loss: 0.6971 - val_binary_accuracy: 0.5103
Epoch 2/30
17/17 [==============================] - 0s 7ms/step - loss: 0.6932 - binary_accuracy: 0.5251 - val_loss: 0.6974 - val_binary_accuracy: 0.4759
Epoch 3/30
17/17 [==============================] - 0s 4ms/step - loss: 0.6916 - binary_accuracy: 0.5290 - val_loss: 0.6975 - val_binary_accuracy: 0.4759
Epoch 4/30
17/17 [==============================] - 0s 5ms/step - loss: 0.6904 - binary_accuracy: 0.5386 - val_loss: 0.6967 - val_binary_accuracy: 0.4897
Epoch 5/30
17/17 [==============================] - 0s 4ms/step - loss: 0.6893 - binary_accuracy: 0.5579 - val_loss: 0.6973 - val_binary_accuracy: 0.4828
Epoch 6/30
17/17 [==============================] - 0s 5ms/step - loss: 0.6878 - binary_accuracy: 0.5734 - val_loss: 0.6975 - val_binary_accuracy: 0.4966
Epoch 7/30
17/17 [==============================] - 0s 6ms/step - loss: 0.6867 - binar

In [41]:
y_hat3_4 = model3_4.predict(X_test_kb_55)
y_hat3_4 = y_hat3_4 > .5
metrics.accuracy_score(test_5w,y_hat3_4)

0.5285714285714286

### Model 4, adding conv layer after pooling

In [42]:
# Model 4_1 - 4_4, adding conv layer after pooling
n_steps = X_train_kb_10.shape[1]
n_features = X_train_kb_10.shape[2]

model4_1 = Sequential()
model4_1.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model4_1.add(Conv1D(filters=8, kernel_size=2, activation='relu')) 
model4_1.add(MaxPooling1D(pool_size=2))
model4_1.add(Conv1D(filters=16, kernel_size=1, activation='relu'))
model4_1.add(Flatten())
model4_1.add(Dense(8, activation='relu')) 
model4_1.add(Dense(1, activation='sigmoid'))

model4_1.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model4_1.fit(X_train_kb_10, train_5w,epochs=30,  validation_data=(X_val_kb_10, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


17/17 [==============================] - 1s 18ms/step - loss: 0.6943 - binary_accuracy: 0.4961 - val_loss: 0.6886 - val_binary_accuracy: 0.5448
Epoch 2/30
17/17 [==============================] - 0s 10ms/step - loss: 0.6932 - binary_accuracy: 0.4961 - val_loss: 0.6896 - val_binary_accuracy: 0.5793
Epoch 3/30
17/17 [==============================] - 0s 4ms/step - loss: 0.6921 - binary_accuracy: 0.4942 - val_loss: 0.6896 - val_binary_accuracy: 0.5586
Epoch 4/30
17/17 [==============================] - 0s 6ms/step - loss: 0.6916 - binary_accuracy: 0.5154 - val_loss: 0.6894 - val_binary_accuracy: 0.5517
Epoch 5/30
17/17 [==============================] - 0s 9ms/step - loss: 0.6912 - binary_accuracy: 0.5135 - val_loss: 0.6895 - val_binary_accuracy: 0.5517


In [43]:
y_hat4_1 = model4_1.predict(X_test_kb_10)
y_hat4_1 = y_hat4_1 > .5
metrics.accuracy_score(test_5w,y_hat4_1)

0.4

In [44]:
n_steps = X_train_kb_25.shape[1]
n_features = X_train_kb_25.shape[2]

model4_2 = Sequential()
model4_2.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model4_2.add(Conv1D(filters=8, kernel_size=2, activation='relu')) 
model4_2.add(MaxPooling1D(pool_size=2))
model4_2.add(Conv1D(filters=16, kernel_size=1, activation='relu'))
model4_2.add(Flatten())
model4_2.add(Dense(25, activation='relu')) 
model4_2.add(Dense(1, activation='sigmoid'))

model4_2.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model4_2.fit(X_train_kb_25, train_5w,epochs=30,  validation_data=(X_val_kb_25, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


17/17 [==============================] - 1s 16ms/step - loss: 0.6965 - binary_accuracy: 0.4884 - val_loss: 0.7006 - val_binary_accuracy: 0.4138
Epoch 2/30
17/17 [==============================] - 0s 7ms/step - loss: 0.6914 - binary_accuracy: 0.5251 - val_loss: 0.6990 - val_binary_accuracy: 0.4138
Epoch 3/30
17/17 [==============================] - 0s 5ms/step - loss: 0.6892 - binary_accuracy: 0.5849 - val_loss: 0.7011 - val_binary_accuracy: 0.4069
Epoch 4/30
17/17 [==============================] - 0s 4ms/step - loss: 0.6870 - binary_accuracy: 0.5637 - val_loss: 0.7022 - val_binary_accuracy: 0.3931
Epoch 5/30
17/17 [==============================] - 0s 7ms/step - loss: 0.6846 - binary_accuracy: 0.6139 - val_loss: 0.7018 - val_binary_accuracy: 0.4069
Epoch 6/30
17/17 [==============================] - 0s 5ms/step - loss: 0.6806 - binary_accuracy: 0.6448 - val_loss: 0.7041 - val_binary_accuracy: 0.4207


In [45]:
y_hat4_2 = model4_2.predict(X_test_kb_25)
y_hat4_2 = y_hat4_2 > .5
metrics.accuracy_score(test_5w,y_hat4_2)

0.5285714285714286

In [46]:
n_steps = X_train_kb_40.shape[1]
n_features = X_train_kb_40.shape[2]

model4_3 = Sequential()
model4_3.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model4_3.add(Conv1D(filters=8, kernel_size=2, activation='relu')) 
model4_3.add(MaxPooling1D(pool_size=2))
model4_3.add(Conv1D(filters=16, kernel_size=1, activation='relu'))
model4_3.add(Flatten())
model4_3.add(Dense(25, activation='relu')) 
model4_3.add(Dense(1, activation='sigmoid'))

model4_3.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model4_3.fit(X_train_kb_40, train_5w,epochs=30,  validation_data=(X_val_kb_40, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


17/17 [==============================] - 1s 16ms/step - loss: 0.6942 - binary_accuracy: 0.4768 - val_loss: 0.6941 - val_binary_accuracy: 0.4759
Epoch 2/30
17/17 [==============================] - 0s 5ms/step - loss: 0.6907 - binary_accuracy: 0.5444 - val_loss: 0.6952 - val_binary_accuracy: 0.4276
Epoch 3/30
17/17 [==============================] - 0s 5ms/step - loss: 0.6884 - binary_accuracy: 0.5849 - val_loss: 0.6960 - val_binary_accuracy: 0.4621
Epoch 4/30
17/17 [==============================] - 0s 7ms/step - loss: 0.6849 - binary_accuracy: 0.6023 - val_loss: 0.6974 - val_binary_accuracy: 0.4345
Epoch 5/30
17/17 [==============================] - 0s 4ms/step - loss: 0.6814 - binary_accuracy: 0.5946 - val_loss: 0.6984 - val_binary_accuracy: 0.4966


In [47]:
y_hat4_3 = model4_3.predict(X_test_kb_40)
y_hat4_3 = y_hat4_3 > .5
metrics.accuracy_score(test_5w,y_hat4_3)

0.45714285714285713

In [48]:
n_steps = X_train_kb_55.shape[1]
n_features = X_train_kb_55.shape[2]

model4_4 = Sequential()
model4_4.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model4_4.add(Conv1D(filters=8, kernel_size=2, activation='relu')) 
model4_4.add(MaxPooling1D(pool_size=2))
model4_4.add(Conv1D(filters=16, kernel_size=1, activation='relu'))
model4_4.add(Flatten())
model4_4.add(Dense(25, activation='relu')) 
model4_4.add(Dense(1, activation='sigmoid'))

model4_4.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model4_4.fit(X_train_kb_55, train_5w,epochs=30,  validation_data=(X_val_kb_55, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


17/17 [==============================] - 1s 12ms/step - loss: 0.7004 - binary_accuracy: 0.5019 - val_loss: 0.6954 - val_binary_accuracy: 0.5103
Epoch 2/30
17/17 [==============================] - 0s 8ms/step - loss: 0.6923 - binary_accuracy: 0.5174 - val_loss: 0.6943 - val_binary_accuracy: 0.4207
Epoch 3/30
17/17 [==============================] - 0s 8ms/step - loss: 0.6900 - binary_accuracy: 0.5502 - val_loss: 0.6942 - val_binary_accuracy: 0.4621
Epoch 4/30
17/17 [==============================] - 0s 4ms/step - loss: 0.6890 - binary_accuracy: 0.5386 - val_loss: 0.6949 - val_binary_accuracy: 0.4483
Epoch 5/30
17/17 [==============================] - 0s 10ms/step - loss: 0.6871 - binary_accuracy: 0.5618 - val_loss: 0.6950 - val_binary_accuracy: 0.4759
Epoch 6/30
17/17 [==============================] - 0s 7ms/step - loss: 0.6859 - binary_accuracy: 0.5618 - val_loss: 0.6952 - val_binary_accuracy: 0.4552
Epoch 7/30
17/17 [==============================] - 0s 7ms/step - loss: 0.6844 - bina

In [49]:
y_hat4_4 = model4_4.predict(X_test_kb_55)
y_hat4_4 = y_hat4_4 > .5
metrics.accuracy_score(test_5w,y_hat4_4)

0.5285714285714286

# Trying the Correlation Method

In [50]:
corr_matrix = X_train.corr()
corr_matrix

feats_corr_10 = list(pd.DataFrame(corr_matrix['target_3'].sort_values(ascending=False).iloc[1:11]).reset_index()['index'])
feats_corr_25 = list(pd.DataFrame(corr_matrix['target_3'].sort_values(ascending=False).iloc[1:26]).reset_index()['index'])
feats_corr_40 = list(pd.DataFrame(corr_matrix['target_3'].sort_values(ascending=False).iloc[1:41]).reset_index()['index'])
feats_corr_55 = list(pd.DataFrame(corr_matrix['target_3'].sort_values(ascending=False).iloc[1:56]).reset_index()['index'])

X_train_cr_10 = X_train[feats_corr_10]
X_test_cr_10 = X_test[feats_corr_10]
X_val_cr_10 = X_val[feats_corr_10]
X_train_cr_25 = X_train[feats_corr_25]
X_test_cr_25 = X_test[feats_corr_25]
X_val_cr_25 = X_val[feats_corr_25]
X_train_cr_40 = X_train[feats_corr_40]
X_test_cr_40 = X_test[feats_corr_40]
X_val_cr_40 = X_val[feats_corr_40]
X_train_cr_55 = X_train[feats_corr_55]
X_test_cr_55 = X_test[feats_corr_55]
X_val_cr_55 = X_val[feats_corr_55]

def pca_finder(df, df_val, df_test):
    
    pca = PCA().fit(df)
    pca_scores = np.cumsum(pca.explained_variance_ratio_) * 100
    res = next(x for x, val in enumerate(pca_scores) if val > 85)
    res = res + 1
    res
    
    pca_cr_1 = PCA(n_components = res).fit(df.to_numpy())
    df = pca_cr_1.transform(df.to_numpy())
    df_val = pca_cr_1.transform(df_val.to_numpy())
    df_test = pca_cr_1.transform(df_test.to_numpy())
    return df, df_val, df_test

X_train_cr_10, X_val_cr_10, X_test_cr_10 = pca_finder(X_train_cr_10, X_val_cr_10, X_test_cr_10)
X_train_cr_25, X_val_cr_25, X_test_cr_25 = pca_finder(X_train_cr_25, X_val_cr_25, X_test_cr_25)
X_train_cr_40, X_val_cr_40, X_test_cr_40 = pca_finder(X_train_cr_40, X_val_cr_40, X_test_cr_40)
X_train_cr_55, X_val_cr_55, X_test_cr_55 = pca_finder(X_train_cr_55, X_val_cr_55, X_test_cr_55)

In [51]:
X_train_cr_40, train_5w = df_to_X_y2(X_train_cr_40,y_train)
X_val_cr_40, val_5w = df_to_X_y2(X_val_cr_40, y_val)
X_test_cr_40, test_5w = df_to_X_y2(X_test_cr_40,y_test) 

X_train_cr_10, _ = df_to_X_y2(X_train_cr_10,y_train)
X_val_cr_10, _ = df_to_X_y2(X_val_cr_10, y_val)
X_test_cr_10, _ = df_to_X_y2(X_test_cr_10,y_test) 

X_train_cr_25, _ = df_to_X_y2(X_train_cr_25,y_train)
X_val_cr_25, _ = df_to_X_y2(X_val_cr_25, y_val)
X_test_cr_25, _ = df_to_X_y2(X_test_cr_25,y_test) 

X_train_cr_55, _ = df_to_X_y2(X_train_cr_55,y_train)
X_val_cr_55, _ = df_to_X_y2(X_val_cr_55, y_val)
X_test_cr_55, _ = df_to_X_y2(X_test_cr_55,y_test) 

In [52]:
# Model 2_1 - 2_4, using differnt k best pca variables. reduced filters
n_steps = X_train_cr_10.shape[1]
n_features = X_train_cr_10.shape[2]

model2_1_cr = Sequential()
model2_1_cr.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model2_1_cr.add(Flatten())
model2_1_cr.add(Dense(25, activation='relu')) 
model2_1_cr.add(Dense(1, activation='sigmoid'))

model2_1_cr.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model2_1_cr.fit(X_train_cr_10, train_5w,epochs=30,  validation_data=(X_val_cr_10, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


17/17 [==============================] - 1s 15ms/step - loss: 0.7032 - binary_accuracy: 0.5019 - val_loss: 0.6796 - val_binary_accuracy: 0.5724
Epoch 2/30
17/17 [==============================] - 0s 5ms/step - loss: 0.6976 - binary_accuracy: 0.5077 - val_loss: 0.6811 - val_binary_accuracy: 0.5586
Epoch 3/30
17/17 [==============================] - 0s 5ms/step - loss: 0.6948 - binary_accuracy: 0.5077 - val_loss: 0.6809 - val_binary_accuracy: 0.5724
Epoch 4/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6923 - binary_accuracy: 0.5290 - val_loss: 0.6822 - val_binary_accuracy: 0.5655
Epoch 5/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6905 - binary_accuracy: 0.5290 - val_loss: 0.6828 - val_binary_accuracy: 0.5379


In [53]:
y_hat2_1_cr = model2_1_cr.predict(X_test_cr_10)
y_hat2_1_cr = y_hat2_1_cr > .5
metrics.accuracy_score(test_5w,y_hat2_1_cr)

0.44285714285714284

In [54]:
n_steps = X_train_cr_25.shape[1]
n_features = X_train_cr_25.shape[2]

model2_2_cr = Sequential()
model2_2_cr.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model2_2_cr.add(Flatten())
model2_2_cr.add(Dense(25, activation='relu')) 
model2_2_cr.add(Dense(1, activation='sigmoid'))

model2_2_cr.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model2_2_cr.fit(X_train_cr_25, train_5w,epochs=30,  validation_data=(X_val_cr_25, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


17/17 [==============================] - 1s 9ms/step - loss: 0.7300 - binary_accuracy: 0.4633 - val_loss: 0.7023 - val_binary_accuracy: 0.4897
Epoch 2/30
17/17 [==============================] - 0s 3ms/step - loss: 0.7121 - binary_accuracy: 0.5039 - val_loss: 0.6966 - val_binary_accuracy: 0.5103
Epoch 3/30
17/17 [==============================] - 0s 3ms/step - loss: 0.7037 - binary_accuracy: 0.5019 - val_loss: 0.6946 - val_binary_accuracy: 0.5241
Epoch 4/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6986 - binary_accuracy: 0.5097 - val_loss: 0.6930 - val_binary_accuracy: 0.5448
Epoch 5/30
17/17 [==============================] - 0s 4ms/step - loss: 0.6940 - binary_accuracy: 0.5154 - val_loss: 0.6936 - val_binary_accuracy: 0.5586
Epoch 6/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6904 - binary_accuracy: 0.5444 - val_loss: 0.6925 - val_binary_accuracy: 0.5586
Epoch 7/30
17/17 [==============================] - 0s 4ms/step - loss: 0.6873 - binary

In [55]:
y_hat2_2_cr = model2_2_cr.predict(X_test_cr_25)
y_hat2_2_cr = y_hat2_2_cr > .5
metrics.accuracy_score(test_5w,y_hat2_2_cr)

0.42857142857142855

In [56]:
n_steps = X_train_cr_40.shape[1]
n_features = X_train_cr_40.shape[2]

model2_3_cr = Sequential()
model2_3_cr.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model2_3_cr.add(Flatten())
model2_3_cr.add(Dense(25, activation='relu')) 
model2_3_cr.add(Dense(1, activation='sigmoid'))

model2_3_cr.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model2_3_cr.fit(X_train_cr_40, train_5w,epochs=30,  validation_data=(X_val_cr_40, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


17/17 [==============================] - 1s 9ms/step - loss: 0.7067 - binary_accuracy: 0.5039 - val_loss: 0.7291 - val_binary_accuracy: 0.4483
Epoch 2/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6935 - binary_accuracy: 0.5290 - val_loss: 0.7245 - val_binary_accuracy: 0.4759
Epoch 3/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6872 - binary_accuracy: 0.5386 - val_loss: 0.7213 - val_binary_accuracy: 0.4690
Epoch 4/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6820 - binary_accuracy: 0.5541 - val_loss: 0.7179 - val_binary_accuracy: 0.4828
Epoch 5/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6766 - binary_accuracy: 0.5734 - val_loss: 0.7147 - val_binary_accuracy: 0.4690
Epoch 6/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6727 - binary_accuracy: 0.5907 - val_loss: 0.7162 - val_binary_accuracy: 0.4897
Epoch 7/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6679 - binary

In [57]:
y_hat2_3_cr = model2_3_cr.predict(X_test_cr_40)
y_hat2_3_cr = y_hat2_3_cr > .5
metrics.accuracy_score(test_5w,y_hat2_3_cr)

0.4142857142857143

In [58]:
n_steps = X_train_cr_55.shape[1]
n_features = X_train_cr_55.shape[2]

model2_4_cr = Sequential()
model2_4_cr.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model2_4_cr.add(Flatten())
model2_4_cr.add(Dense(25, activation='relu')) 
model2_4_cr.add(Dense(1, activation='sigmoid'))

model2_4_cr.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model2_4_cr.fit(X_train_cr_55, train_5w,epochs=30,  validation_data=(X_val_cr_55, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


17/17 [==============================] - 1s 10ms/step - loss: 0.7641 - binary_accuracy: 0.5019 - val_loss: 0.7109 - val_binary_accuracy: 0.5379
Epoch 2/30
17/17 [==============================] - 0s 3ms/step - loss: 0.7194 - binary_accuracy: 0.5077 - val_loss: 0.6942 - val_binary_accuracy: 0.4966
Epoch 3/30
17/17 [==============================] - 0s 3ms/step - loss: 0.7054 - binary_accuracy: 0.4923 - val_loss: 0.6933 - val_binary_accuracy: 0.4759
Epoch 4/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6989 - binary_accuracy: 0.4865 - val_loss: 0.6938 - val_binary_accuracy: 0.4897
Epoch 5/30
17/17 [==============================] - 0s 4ms/step - loss: 0.6951 - binary_accuracy: 0.4942 - val_loss: 0.6928 - val_binary_accuracy: 0.4483
Epoch 6/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6907 - binary_accuracy: 0.5116 - val_loss: 0.6923 - val_binary_accuracy: 0.4483
Epoch 7/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6869 - binar

In [59]:
y_hat2_4_cr = model2_4_cr.predict(X_test_cr_55)
y_hat2_4_cr = y_hat2_4_cr > .5
metrics.accuracy_score(test_5w,y_hat2_4_cr)

0.45714285714285713

In [60]:
# Model 3_1 - 3_4, adding layers
n_steps = X_train_cr_10.shape[1]
n_features = X_train_cr_10.shape[2]

model3_1_cr = Sequential()
model3_1_cr.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model3_1_cr.add(Conv1D(filters=8, kernel_size=2, activation='relu')) 
model3_1_cr.add(MaxPooling1D(pool_size=2))
model3_1_cr.add(Flatten())
model3_1_cr.add(Dense(8, activation='relu')) 
model3_1_cr.add(Dense(1, activation='sigmoid'))

model3_1_cr.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model3_1_cr.fit(X_train_cr_10, train_5w,epochs=30,  validation_data=(X_val_cr_10, val_5w), callbacks = [early_stopping_monitor])
y_hat3_1_cr = model3_1_cr.predict(X_test_cr_10)
y_hat3_1_cr = y_hat3_1_cr > .5
metrics.accuracy_score(test_5w,y_hat3_1_cr)


Epoch 1/30


/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


17/17 [==============================] - 1s 17ms/step - loss: 0.6948 - binary_accuracy: 0.4595 - val_loss: 0.6968 - val_binary_accuracy: 0.5034
Epoch 2/30
17/17 [==============================] - 0s 4ms/step - loss: 0.6936 - binary_accuracy: 0.4826 - val_loss: 0.6971 - val_binary_accuracy: 0.4414
Epoch 3/30
17/17 [==============================] - 0s 4ms/step - loss: 0.6921 - binary_accuracy: 0.5328 - val_loss: 0.6961 - val_binary_accuracy: 0.4690
Epoch 4/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6912 - binary_accuracy: 0.5347 - val_loss: 0.6958 - val_binary_accuracy: 0.5034
Epoch 5/30
17/17 [==============================] - 0s 19ms/step - loss: 0.6903 - binary_accuracy: 0.5444 - val_loss: 0.6959 - val_binary_accuracy: 0.4897
Epoch 6/30
17/17 [==============================] - 0s 9ms/step - loss: 0.6901 - binary_accuracy: 0.5502 - val_loss: 0.6964 - val_binary_accuracy: 0.4828
Epoch 7/30
17/17 [==============================] - 0s 7ms/step - loss: 0.6893 - bina

0.42857142857142855

In [61]:
n_steps = X_train_cr_25.shape[1]
n_features = X_train_cr_25.shape[2]

model3_2_cr = Sequential()
model3_2_cr.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model3_2_cr.add(Conv1D(filters=8, kernel_size=2, activation='relu')) 
model3_2_cr.add(MaxPooling1D(pool_size=2))
model3_2_cr.add(Flatten())
model3_2_cr.add(Dense(25, activation='relu')) 
model3_2_cr.add(Dense(1, activation='sigmoid'))

model3_2_cr.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model3_2_cr.fit(X_train_cr_25, train_5w,epochs=30,  validation_data=(X_val_cr_25, val_5w), callbacks = [early_stopping_monitor])
y_hat3_2_cr = model3_2_cr.predict(X_test_cr_25)
y_hat3_2_cr = y_hat3_2_cr > .5
metrics.accuracy_score(test_5w,y_hat3_2_cr)


Epoch 1/30


/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


17/17 [==============================] - 1s 10ms/step - loss: 0.6939 - binary_accuracy: 0.4942 - val_loss: 0.6952 - val_binary_accuracy: 0.4345
Epoch 2/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6897 - binary_accuracy: 0.5772 - val_loss: 0.6961 - val_binary_accuracy: 0.4276
Epoch 3/30
17/17 [==============================] - 0s 5ms/step - loss: 0.6872 - binary_accuracy: 0.6158 - val_loss: 0.6951 - val_binary_accuracy: 0.4621
Epoch 4/30
17/17 [==============================] - 0s 5ms/step - loss: 0.6844 - binary_accuracy: 0.6236 - val_loss: 0.6946 - val_binary_accuracy: 0.4966
Epoch 5/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6822 - binary_accuracy: 0.6409 - val_loss: 0.6948 - val_binary_accuracy: 0.4966
Epoch 6/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6791 - binary_accuracy: 0.6583 - val_loss: 0.6954 - val_binary_accuracy: 0.4966
Epoch 7/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6756 - binar

0.5428571428571428

In [62]:

n_steps = X_train_cr_40.shape[1]
n_features = X_train_cr_40.shape[2]

model3_3_cr = Sequential()
model3_3_cr.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model3_3_cr.add(Conv1D(filters=8, kernel_size=2, activation='relu')) 
model3_3_cr.add(MaxPooling1D(pool_size=2))
model3_3_cr.add(Flatten())
model3_3_cr.add(Dense(25, activation='relu')) 
model3_3_cr.add(Dense(1, activation='sigmoid'))

model3_3_cr.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model3_3_cr.fit(X_train_cr_40, train_5w,epochs=30,  validation_data=(X_val_cr_40, val_5w), callbacks = [early_stopping_monitor])
y_hat3_3_cr = model3_3_cr.predict(X_test_cr_40)
y_hat3_3_cr = y_hat3_3_cr > .5
metrics.accuracy_score(test_5w,y_hat3_3_cr)


Epoch 1/30


/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


17/17 [==============================] - 1s 11ms/step - loss: 0.7003 - binary_accuracy: 0.4846 - val_loss: 0.6944 - val_binary_accuracy: 0.5034
Epoch 2/30
17/17 [==============================] - 0s 4ms/step - loss: 0.6924 - binary_accuracy: 0.5309 - val_loss: 0.6936 - val_binary_accuracy: 0.5172
Epoch 3/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6885 - binary_accuracy: 0.5618 - val_loss: 0.6927 - val_binary_accuracy: 0.5310
Epoch 4/30
17/17 [==============================] - 0s 4ms/step - loss: 0.6846 - binary_accuracy: 0.6004 - val_loss: 0.6908 - val_binary_accuracy: 0.5517
Epoch 5/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6815 - binary_accuracy: 0.6216 - val_loss: 0.6901 - val_binary_accuracy: 0.5655
Epoch 6/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6769 - binary_accuracy: 0.6216 - val_loss: 0.6912 - val_binary_accuracy: 0.5862
Epoch 7/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6703 - binar

0.4

In [63]:
n_steps = X_train_cr_55.shape[1]
n_features = X_train_cr_55.shape[2]

model3_4_cr = Sequential()
model3_4_cr.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model3_4_cr.add(Conv1D(filters=8, kernel_size=2, activation='relu')) 
model3_4_cr.add(MaxPooling1D(pool_size=2))
model3_4_cr.add(Flatten())
model3_4_cr.add(Dense(25, activation='relu')) 
model3_4_cr.add(Dense(1, activation='sigmoid'))

model3_4_cr.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model3_4_cr.fit(X_train_cr_55, train_5w,epochs=30,  validation_data=(X_val_cr_55, val_5w), callbacks = [early_stopping_monitor])
y_hat3_4_cr = model3_4_cr.predict(X_test_cr_55)
y_hat3_4_cr = y_hat3_4_cr > .5
metrics.accuracy_score(test_5w,y_hat3_4_cr)

Epoch 1/30


/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


17/17 [==============================] - 1s 14ms/step - loss: 0.7292 - binary_accuracy: 0.4961 - val_loss: 0.7089 - val_binary_accuracy: 0.5310
Epoch 2/30
17/17 [==============================] - 0s 3ms/step - loss: 0.7060 - binary_accuracy: 0.4768 - val_loss: 0.6995 - val_binary_accuracy: 0.4759
Epoch 3/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6987 - binary_accuracy: 0.4865 - val_loss: 0.6986 - val_binary_accuracy: 0.4690
Epoch 4/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6962 - binary_accuracy: 0.5019 - val_loss: 0.6986 - val_binary_accuracy: 0.4552
Epoch 5/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6948 - binary_accuracy: 0.5058 - val_loss: 0.6981 - val_binary_accuracy: 0.4552
Epoch 6/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6930 - binary_accuracy: 0.5116 - val_loss: 0.6975 - val_binary_accuracy: 0.4414
Epoch 7/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6915 - binar

0.5428571428571428

# Changes to the Target Window

In [64]:
X_train_kb_10, X_val_kb_10, X_test_kb_10 = kbest_creator(10, X_train.drop('target_3', axis=1), X_val.drop('target_3', axis=1), X_test.drop('target_3', axis=1)) 
X_train_kb_25, X_val_kb_25, X_test_kb_25 = kbest_creator(25, X_train.drop('target_3', axis=1), X_val.drop('target_3', axis=1), X_test.drop('target_3', axis=1)) 
X_train_kb_40, X_val_kb_40, X_test_kb_40 = kbest_creator(40, X_train.drop('target_3', axis=1), X_val.drop('target_3', axis=1), X_test.drop('target_3', axis=1)) 
X_train_kb_55, X_val_kb_55, X_test_kb_55 = kbest_creator(55, X_train.drop('target_3', axis=1), X_val.drop('target_3', axis=1), X_test.drop('target_3', axis=1)) 

/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:301: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:301: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:301: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:301: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms


In [65]:
def df_to_X_y_mod(df, target, window_size=5):
  # converts to matrix of numpy arrays
  X = []
  y = []
  for i in range(len(df)-window_size): # length of data frame - window_size so it does't take empty values at the end, 
    # does force you to loose the last 5 values, could fix with padding
    row = [r for r in df[i:i+window_size]] # grabs row i and all rows above within the window size length
    X.append(row) # creates 3 dimentional array, (# obseravtions, # rows in window, # features)
    label = target[i+window_size-1] # pulls the target variable after the window, target varible needs to be column zero in this 
    y.append(label) # returns (N,) martix of targets i+window_length time periods away
  return np.array(X), np.array(y)
  


In [66]:
# converting to window format, in this case 5 periods
X_train_kb_40, train_5w = df_to_X_y2(X_train_kb_40,y_train)
X_val_kb_40, val_5w = df_to_X_y2(X_val_kb_40, y_val)
X_test_kb_40, test_5w = df_to_X_y2(X_test_kb_40,y_test) 

X_train_kb_10, _ = df_to_X_y2(X_train_kb_10,y_train)
X_val_kb_10, _ = df_to_X_y2(X_val_kb_10, y_val)
X_test_kb_10, _ = df_to_X_y2(X_test_kb_10,y_test) 

X_train_kb_25, _ = df_to_X_y2(X_train_kb_25,y_train)
X_val_kb_25, _ = df_to_X_y2(X_val_kb_25, y_val)
X_test_kb_25, _ = df_to_X_y2(X_test_kb_25,y_test) 

X_train_kb_55, _ = df_to_X_y2(X_train_kb_55,y_train)
X_val_kb_55, _ = df_to_X_y2(X_val_kb_55, y_val)
X_test_kb_55, _ = df_to_X_y2(X_test_kb_55,y_test) 

In [67]:
n_steps = X_train_kb_25.shape[1]
n_features = X_train_kb_25.shape[2]

model_mod4 = Sequential()
model_mod4.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model_mod4.add(Conv1D(filters=8, kernel_size=2, activation='relu')) 
model_mod4.add(MaxPooling1D(pool_size=2))
model_mod4.add(Conv1D(filters=16, kernel_size=1, activation='relu'))
model_mod4.add(Flatten())
model_mod4.add(Dense(25, activation='relu')) 
model_mod4.add(Dense(1, activation='sigmoid'))

model_mod4.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model_mod4.fit(X_train_kb_25, train_5w,epochs=30,  validation_data=(X_val_kb_25, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


17/17 [==============================] - 1s 19ms/step - loss: 0.6974 - binary_accuracy: 0.5000 - val_loss: 0.6860 - val_binary_accuracy: 0.5241
Epoch 2/30
17/17 [==============================] - 0s 5ms/step - loss: 0.6921 - binary_accuracy: 0.4942 - val_loss: 0.6868 - val_binary_accuracy: 0.5310
Epoch 3/30
17/17 [==============================] - 0s 6ms/step - loss: 0.6906 - binary_accuracy: 0.5309 - val_loss: 0.6870 - val_binary_accuracy: 0.5517
Epoch 4/30
17/17 [==============================] - 0s 6ms/step - loss: 0.6889 - binary_accuracy: 0.5367 - val_loss: 0.6861 - val_binary_accuracy: 0.5103
Epoch 5/30
17/17 [==============================] - 0s 5ms/step - loss: 0.6878 - binary_accuracy: 0.5463 - val_loss: 0.6861 - val_binary_accuracy: 0.5172


In [68]:
y_hat_mod4_kb = model_mod4.predict(X_test_kb_25)
y_hat_mod4_kb = y_hat_mod4_kb > .5
metrics.accuracy_score(test_5w,y_hat_mod4_kb)

0.42857142857142855

In [69]:
model_mod3 = Sequential()
model_mod3.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model_mod3.add(Conv1D(filters=8, kernel_size=2, activation='relu')) 
model_mod3.add(MaxPooling1D(pool_size=2))
model_mod3.add(Flatten())
model_mod3.add(Dense(25, activation='relu')) 
model_mod3.add(Dense(1, activation='sigmoid'))

model_mod3.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model_mod3.fit(X_train_kb_25, train_5w,epochs=30,  validation_data=(X_val_kb_25, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


17/17 [==============================] - 1s 12ms/step - loss: 0.6984 - binary_accuracy: 0.5058 - val_loss: 0.6932 - val_binary_accuracy: 0.4966
Epoch 2/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6929 - binary_accuracy: 0.5386 - val_loss: 0.6934 - val_binary_accuracy: 0.4966
Epoch 3/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6907 - binary_accuracy: 0.5541 - val_loss: 0.6931 - val_binary_accuracy: 0.5793
Epoch 4/30
17/17 [==============================] - 0s 11ms/step - loss: 0.6886 - binary_accuracy: 0.5483 - val_loss: 0.6954 - val_binary_accuracy: 0.5448
Epoch 5/30
17/17 [==============================] - 0s 7ms/step - loss: 0.6869 - binary_accuracy: 0.5618 - val_loss: 0.6950 - val_binary_accuracy: 0.5310
Epoch 6/30
17/17 [==============================] - 0s 7ms/step - loss: 0.6855 - binary_accuracy: 0.5695 - val_loss: 0.6964 - val_binary_accuracy: 0.5448
Epoch 7/30
17/17 [==============================] - 0s 6ms/step - loss: 0.6844 - bina

In [70]:
y_hat_mod3_kb = model_mod3.predict(X_test_kb_25)
y_hat_mod3_kb = y_hat_mod3_kb > .5
metrics.accuracy_score(test_5w,y_hat_mod3_kb)

0.5571428571428572

In [71]:
model_mod2 = Sequential()
model_mod2.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model_mod2.add(Conv1D(filters=8, kernel_size=2, activation='relu')) 
model_mod2.add(Flatten())
model_mod2.add(Dense(25, activation='relu')) 
model_mod2.add(Dense(1, activation='sigmoid'))

model_mod2.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model_mod2.fit(X_train_kb_25, train_5w,epochs=30,  validation_data=(X_val_kb_25, val_5w))

Epoch 1/30


/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


17/17 [==============================] - 1s 15ms/step - loss: 0.6942 - binary_accuracy: 0.5000 - val_loss: 0.7006 - val_binary_accuracy: 0.5379
Epoch 2/30
17/17 [==============================] - 0s 6ms/step - loss: 0.6903 - binary_accuracy: 0.5174 - val_loss: 0.6993 - val_binary_accuracy: 0.5034
Epoch 3/30
17/17 [==============================] - 0s 10ms/step - loss: 0.6856 - binary_accuracy: 0.5541 - val_loss: 0.6998 - val_binary_accuracy: 0.4828
Epoch 4/30
17/17 [==============================] - 0s 7ms/step - loss: 0.6842 - binary_accuracy: 0.5579 - val_loss: 0.6999 - val_binary_accuracy: 0.4966
Epoch 5/30
17/17 [==============================] - 0s 6ms/step - loss: 0.6818 - binary_accuracy: 0.5598 - val_loss: 0.7002 - val_binary_accuracy: 0.4966
Epoch 6/30
17/17 [==============================] - 0s 6ms/step - loss: 0.6800 - binary_accuracy: 0.5714 - val_loss: 0.7000 - val_binary_accuracy: 0.5103
Epoch 7/30
17/17 [==============================] - 0s 4ms/step - loss: 0.6780 - bina

In [72]:
y_hat_mod2_kb = model_mod2.predict(X_test_kb_25)
y_hat_mod2_kb = y_hat_mod2_kb > .5
metrics.accuracy_score(test_5w,y_hat_mod2_kb)

0.5142857142857142

In [73]:
model_mod1 = Sequential()
model_mod1.add(Conv1D(filters=8, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model_mod1.add(Flatten())
model_mod1.add(Dense(25, activation='relu')) 
model_mod1.add(Dense(1, activation='sigmoid'))

model_mod1.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model_mod1.fit(X_train_kb_25, train_5w,epochs=30,  validation_data=(X_val_kb_25, val_5w))

Epoch 1/30


/Users/fabrizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


17/17 [==============================] - 1s 9ms/step - loss: 0.7142 - binary_accuracy: 0.4575 - val_loss: 0.6890 - val_binary_accuracy: 0.5034
Epoch 2/30
17/17 [==============================] - 0s 3ms/step - loss: 0.7077 - binary_accuracy: 0.4826 - val_loss: 0.6881 - val_binary_accuracy: 0.5034
Epoch 3/30
17/17 [==============================] - 0s 3ms/step - loss: 0.7025 - binary_accuracy: 0.4942 - val_loss: 0.6865 - val_binary_accuracy: 0.5379
Epoch 4/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6987 - binary_accuracy: 0.4961 - val_loss: 0.6860 - val_binary_accuracy: 0.5448
Epoch 5/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6954 - binary_accuracy: 0.5154 - val_loss: 0.6860 - val_binary_accuracy: 0.5379
Epoch 6/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6927 - binary_accuracy: 0.5405 - val_loss: 0.6844 - val_binary_accuracy: 0.5655
Epoch 7/30
17/17 [==============================] - 0s 3ms/step - loss: 0.6898 - binary

In [74]:
y_hat_mod1_kb = model_mod1.predict(X_test_kb_25)
y_hat_mod1_kb = y_hat_mod1_kb > .5
metrics.accuracy_score(test_5w,y_hat_mod1_kb)

0.5285714285714286